In [1]:
import matplotlib.pyplot as plt 
import json
import pickle
import torch
import random 
import numpy as np
import os 

from torch.optim import SGD, Adam
from torch.utils.data import DataLoader, random_split
from torch.nn.functional import binary_cross_entropy
from sklearn import metrics 

from data_loaders.assist2009 import ASSIST2009
from data_loaders.assist2012 import ASSIST2012
from data_loaders.ednet01 import EdNet01

from models.dkvmn_text import SUBJ_DKVMN
from models.dkvmn_text import train_model as plus_train

from models.utils import collate_fn, collate_ednet, cal_acc_class

%matplotlib inline

seed = 1004

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

c:\Users\user2\Desktop\master_degree\research\env_research\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'dkvmn+'
dataset_name = 'ASSIST2009'
dataset = None
ckpts = f"ckpts/{model_name}/{dataset_name}/"

with open("config.json") as f:
    config = json.load(f)
    model_config = config[model_name]
    train_config = config["train_config"]
    
batch_size = train_config["batch_size"]
num_epochs = train_config["num_epochs"]
train_ratio = train_config["train_ratio"]
learning_rate = train_config["learning_rate"]
optimizer = train_config["optimizer"] # can be sgd, adam
seq_len = train_config["seq_len"] # 샘플링 할 갯수

In [3]:
# 데이터셋 추가 가능
collate_pt = collate_fn
if dataset_name == "ASSIST2009":
    dataset = ASSIST2009(seq_len, 'datasets/ASSIST2009/')
elif dataset_name == "ASSIST2012":
    dataset = ASSIST2012(seq_len, 'datasets/ASSIST2012/')

In [4]:
def train_model(model, test_loader, ckpt_path):
    '''
        Args:
            train_loader: the PyTorch DataLoader instance for training
            test_loader: the PyTorch DataLoader instance for test
            num_epochs: the number of epochs
            opt: the optimization to train this model
            ckpt_path: the path to save this model's parameters
    '''
    aucs = []
    loss_means = []  
    accs = []
    q_accs = {}
    
    max_auc = 0
    
    # Test
    model.load_state_dict(torch.load(os.path.join(ckpt_path, "model.ckpt"), map_location=device))
    loss_mean = []
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            q, r, qshft_seqs, rshft_seqs, m, bert_s, bert_t, bert_m, q2diff_seqs, pid_seqs, pidshift, hint_seqs = data

            model.eval()

            y, Mv, w = model(q.long(), r.long(), bert_s, bert_t, bert_m, q2diff_seqs.long(), pid_seqs.long())

            # y와 t 변수에 있는 행렬들에서 마스킹이 true로 된 값들만 불러옴
            q = torch.masked_select(q, m).detach().cpu()
            y = torch.masked_select(y, m).detach().cpu()
            t = torch.masked_select(r, m).detach().cpu()

            auc = metrics.roc_auc_score(
                y_true=t.numpy(), y_score=y.numpy()
            )
            bin_y = [1 if p >= 0.5 else 0 for p in y.numpy()]
            acc = metrics.accuracy_score(t.numpy(), bin_y)
            loss = binary_cross_entropy(y, t) # 실제 y^T와 원핫 결합, 다음 answer 간 cross entropy

            print(f"[Test] number: {i}, AUC: {auc}, ACC: :{acc} Loss: {loss} ")

            # evaluation metrics
            aucs.append(auc)
            loss_mean.append(loss)     
            accs.append(acc)
            q_accs, cnt = cal_acc_class(q.long(), t.long(), bin_y)
        loss_means.append(np.mean(loss_mean))


    return aucs, loss_means, accs, q_accs, cnt, Mv, w

In [5]:
model = torch.nn.DataParallel(SUBJ_DKVMN(dataset.num_q, num_qid=dataset.num_pid, **model_config)).to(device)
train_model = train_model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# 데이터셋 분할
data_size = len(dataset)
train_size = int(data_size * train_ratio) 
valid_size = int(data_size * ((1.0 - train_ratio) / 2.0))
test_size = data_size - train_size - valid_size

train_dataset, valid_dataset, test_dataset = random_split(
    dataset, [train_size, valid_size, test_size], generator=torch.Generator(device=device)
)

# pickle에 얼마만큼 분할했는지 읽기
if os.path.exists(os.path.join(dataset.dataset_dir, "train_indices.pkl")):
    with open(
        os.path.join(dataset.dataset_dir, "train_indices.pkl"), "rb"
    ) as f:
        train_dataset.indices = pickle.load(f)
    with open(
        os.path.join(dataset.dataset_dir, "valid_indicies.pkl"), "rb"
    ) as f:
        valid_dataset.indices = pickle.load(f)
    with open(
        os.path.join(dataset.dataset_dir, "test_indices.pkl"), "rb"
    ) as f:
        test_dataset.indices = pickle.load(f)

In [7]:
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    collate_fn=collate_pt, generator=torch.Generator(device=device)
)
valid_loader = DataLoader(
    valid_dataset, batch_size=batch_size, shuffle=True,
    collate_fn=collate_pt, generator=torch.Generator(device=device)
)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True,
    collate_fn=collate_pt, generator=torch.Generator(device=device)
)

if optimizer == "sgd":
    opt = SGD(model.parameters(), learning_rate, momentum=0.9)
elif optimizer == "adam":
    opt = Adam(model.parameters(), learning_rate)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.5)
opt.lr_scheduler = lr_scheduler

# 모델에서 미리 정의한 함수로 AUCS와 LOSS 계산    
aucs, loss_means, accs, q_accs, q_cnts, Mv, w = \
    train_model(
        model, test_loader, ckpts
    )

[Test] number: 0, AUC: 0.9837072343545911, ACC: :0.9469632164242943 Loss: 0.1328725516796112 
[Test] number: 1, AUC: 0.9822715815499222, ACC: :0.9587878787878787 Loss: 0.139503613114357 
[Test] number: 2, AUC: 0.9934436913451512, ACC: :0.9694767441860465 Loss: 0.07316002994775772 
[Test] number: 3, AUC: 0.9889503043888026, ACC: :0.9680851063829787 Loss: 0.08745691925287247 
[Test] number: 4, AUC: 0.9846775343177949, ACC: :0.9506976744186046 Loss: 0.1377621591091156 
[Test] number: 5, AUC: 0.9635718083086504, ACC: :0.9308510638297872 Loss: 0.22788003087043762 
[Test] number: 6, AUC: 0.9745466553811428, ACC: :0.9473684210526315 Loss: 0.17726880311965942 
[Test] number: 7, AUC: 0.9612836438923396, ACC: :0.9347133757961783 Loss: 0.20352251827716827 
[Test] number: 8, AUC: 0.9748021139727862, ACC: :0.9364326375711575 Loss: 0.17721284925937653 
[Test] number: 9, AUC: 0.9704872220444727, ACC: :0.9231916480238628 Loss: 0.22785037755966187 
[Test] number: 10, AUC: 0.9896079377818652, ACC: :0.96

In [16]:
print(torch.min(Mv[:, :-1])) # 컨셉 수 / 시퀀스? 임베딩 수

tensor(-2.2845)


In [20]:
print(w)

tensor([[[1.0816e-03, 3.3317e-01, 3.7621e-04,  ..., 1.9917e-02,
          1.3704e-03, 3.8838e-04],
         [1.0816e-03, 3.3317e-01, 3.7621e-04,  ..., 1.9917e-02,
          1.3704e-03, 3.8838e-04],
         [1.0816e-03, 3.3317e-01, 3.7621e-04,  ..., 1.9917e-02,
          1.3704e-03, 3.8838e-04],
         ...,
         [2.3979e-04, 3.9177e-04, 4.3399e-04,  ..., 5.6806e-06,
          1.1918e-03, 4.2809e-04],
         [2.3979e-04, 3.9177e-04, 4.3399e-04,  ..., 5.6806e-06,
          1.1918e-03, 4.2809e-04],
         [2.3979e-04, 3.9177e-04, 4.3399e-04,  ..., 5.6806e-06,
          1.1918e-03, 4.2809e-04]],

        [[3.6447e-03, 9.1907e-04, 3.4935e-04,  ..., 2.1074e-04,
          7.6429e-04, 1.4949e-03],
         [3.6447e-03, 9.1907e-04, 3.4935e-04,  ..., 2.1074e-04,
          7.6429e-04, 1.4949e-03],
         [5.6119e-04, 4.3690e-03, 1.4877e-04,  ..., 4.8740e-01,
          9.1777e-04, 6.7118e-04],
         ...,
         [1.0042e-03, 7.0250e-04, 1.5304e-03,  ..., 1.4714e-02,
          5.300

In [22]:
# 모든 W값에 대한 빈 텐서를 생성
# 마지막 layer 빼서 넘겨보기

print(w[0][99][49])
print(torch.min((w.unsqueeze(-1) * Mv[:, :-1]).sum(-2)))


tensor(0.0004)
tensor(-1.8045)
